In [1]:
# pip install spotipy --update
# pip install --upgrade library_name
# pip install python-dotenv


## preparation - importing pandas, spotipy etc., reading csv

In [2]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
import json

import os
from dotenv import load_dotenv, dotenv_values


In [3]:
songs = pd.read_csv(
    './data/songs1.csv')


In [4]:
songs.head(2)


,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,id,html,cluster
0,Se Eu Quiser Falar Com Deus ...,Gilberto Gil,0.658,0.259,11,-13.141,0,0.0705,0.694,0.000059,0.975,0.306,110.376,,256213,4,1n7JnwviZ7zf0LR1tcGFq7,https://open.spotify.com/track/1n7JnwviZ7zf0LR...,15
1,Saudade De Bahia ...,Antônio Carlos Jobim,0.742,0.399,2,-12.646,1,0.0346,0.217,0.000002,0.107,0.693,125.039,,191867,4,5QGM1U0eCYrQuwSJwTm5Zq,https://open.spotify.com/track/5QGM1U0eCYrQuwS...,9


## preparing connection to app in spotify

In [5]:
load_dotenv()


Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 7


True

In [6]:
# scope = "user-library-read"
scope = 'playlist-modify-public'
username = os.getenv("spotify_username")
redirectUri = "http://localhost:8000/callback"
# pip install spotipy --update
# pip install --upgrade library_name9',
token = SpotifyOAuth(scope=scope, username=username, client_id=os.getenv("spotify_client_id"),
                     client_secret=os.getenv("spotify_client_secret"), redirect_uri='http://localhost:8000/callback',)
spotifyObject = spotipy.Spotify(auth_manager=token)


In [7]:
songs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5235 entries, 0 to 5234
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              5235 non-null   object 
 1   artist            5235 non-null   object 
 2   danceability      5235 non-null   float64
 3   energy            5235 non-null   float64
 4   key               5235 non-null   int64  
 5   loudness          5235 non-null   float64
 6   mode              5235 non-null   int64  
 7   speechiness       5235 non-null   float64
 8   acousticness      5235 non-null   float64
 9   instrumentalness  5235 non-null   float64
 10  liveness          5235 non-null   float64
 11  valence           5235 non-null   float64
 12  tempo             5235 non-null   float64
 13  type              5235 non-null   object 
 14  duration_ms       5235 non-null   int64  
 15  time_signature    5235 non-null   int64  
 16  id                5235 non-null   object 


# create playlists as list of df

In [8]:
playlists = []
for i in range(21):
    playlists.append(songs[songs['cluster'] == i])


In [9]:
len(playlists)


21

In [12]:
import itertools

for i, playlist in enumerate(playlists):
    playlist_name = f'Playlist{i}'
    playlist_description = ''
    playlist_id = spotifyObject.user_playlist_create(
        user=username, name=playlist_name, public=True, description=playlist_description)['id']
    list_of_songs = []
    for index, song in playlist.iterrows():
        result = spotifyObject.search(
            q=f"{song['name']} artist:{song['artist']}", type='track', limit=1)
        try:
            list_of_songs.append(result['tracks']['items'][0]['id'])
        except IndexError:
            continue

    track_chunks = list(itertools.islice(list_of_songs, 100))
    spotifyObject.user_playlist_add_tracks(
        user=username, playlist_id=playlist_id, tracks=track_chunks)
